Course Instructor: Bernd Neumayr, JKU

# UE05: SHACL

Complete the **10 tasks (1 point per task)** in the `4. SHACL` sheet of `SemAI.jar` first and then transfer them to this notebook.

For each task include:
- A headline including the task number
- The task description 
- The data graph and your solution (the shapes graph) in executable form
- After executing the validation, print out the validation results in tabular form.  




# Preparation



In [16]:
# Install required packages in the current Jupyter kernel
!pip install -q rdflib 
!pip3 install -q pyshacl

In [17]:
# Imports
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace, Dataset
from rdflib.namespace import FOAF , XSD , RDFS 
from rdflib.plugins.sparql.processor import SPARQLResult
from rdflib.namespace import NamespaceManager

from pyshacl import validate

import pandas as pd

def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = [ { var : res[var].n3(graph.namespace_manager) if (isinstance(res[var],URIRef) and use_prefixes) else res[var] for var in results.vars } for res in results ]     
                                        # construct a list of dictionaries, as intermediate format to construct the pandas DataFrame, use prefixes to abbreviate URIs                
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns      

def validation_report_as_dataframe(validation_report):
  df = sparql_select(results_graph,"""
		SELECT  ?focusNode ?resultPath ?value ?sourceConstraintComponent ?sourceShape ?resultMessage
		WHERE
  		{ ?vr	a sh:ValidationResult ;
						sh:focusNode ?focusNode ;
						sh:sourceConstraintComponent ?sourceConstraintComponent ;
						sh:sourceShape ?sourceShape ;
						sh:resultMessage ?resultMessage .					 
				OPTIONAL { ?vr sh:value ?value . }
				OPTIONAL { ?vr sh:resultPath ?resultPath . }
  		}
  """,use_prefixes=True)
  return df

def shacl_validate(dg,sg):
  return validate(dg,shacl_graph=sg,
      inference='rdfs',
      abort_on_first=False,
      allow_infos=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=False,
      js=False,
      debug=False)  
  

def shacl_validate_with_rules(dg,sg):
	return validate(dg,shacl_graph=sg,
      inference='rdfs',
      abort_on_first=False,
      allow_infos=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=True,
      iterate_rules=True, inplace=True,
      js=False,
      debug=False)

# Task 1

Wenn eine Person eine x:knows Beziehung hat, dann sollte das Ziel dieser Beziehung als IRI angegeben sein.
 


In [18]:
sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<KnowsShape> a sh:PropertyShape ;
    sh:targetClass <Person> ;
    sh:path :knows ;
    sh:nodeKind sh:IRI  .
""")

dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<Peter>  rdf:type  <Person> ;
        :knows    <John> ;
        :knows    [ :name  "Peter" ] .

<John>  :knows  [ :name   "Mary" ] .
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)


@prefix : <http://example.org/properties/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Peter> ;
            sh:resultMessage "Value is not of Node Kind sh:IRI" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NodeKindConstraintComponent ;
            sh:sourceShape <http://example.org/entities/KnowsShape> ;
            sh:value [ a rdfs:Resource ;
                    :name "Peter" ] ] .




,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/Peter>,:knows,n2a9cfdd1fa734ccba00af2a10bedb622b1,sh:NodeKindConstraintComponent,<http://example.org/entities/KnowsShape>,Value is not of Node Kind sh:IRI


# Task 2

Die Klassen Man und Woman sind disjunkt. Hinweis: verwenden Sie sh:not.

In [19]:
sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<ManShape> a sh:NodeShape ;
sh:targetClass <Man> ;
sh:class <Man> .

<WomanShape> a sh:NodeShape ;
sh:targetClass <Woman> ;
sh:class <Woman> ;
sh:not <ManShape> .
""")

dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<YoungMan>  rdfs:subClassOf  <Man> .

<John>  rdf:type  <YoungWoman> , <YoungMan> .

<Mary>  rdf:type  <Woman> .

<Peter>  rdf:type  <Man> .

<YoungWoman>  rdfs:subClassOf  <Woman> .

<Jane>  rdf:type  <YoungWoman> .

<Susi>  rdf:type  <YoungWoman> , <Man> .
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Susi> ;
            sh:resultMessage "Node <http://example.org/entities/Susi> conforms to shape <http://example.org/entities/ManShape>" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NotConstraintComponent ;
            sh:sourceShape <http://example.org/entities/WomanShape> ;
            sh:value <http://example.org/entities/Susi> ],
        [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/John> ;
            sh:resultMessage "Node <http://example.org/entities/John> conforms to shape <http://example.org/entities/ManShape>" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NotConstraintComponent ;
            sh:sourceShape <http://exa

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/John>,None,<http://example.org/entities/John>,sh:NotConstraintComponent,<http://example.org/entities/WomanShape>,Node <http://example.org/entities/John> confor...
1,<http://example.org/entities/Susi>,None,<http://example.org/entities/Susi>,sh:NotConstraintComponent,<http://example.org/entities/WomanShape>,Node <http://example.org/entities/Susi> confor...


# Task 3

Wenn eine Person eine :knows Beziehung hat, dann muss das Ziel dieser Beziehung eine Person sein.

In [20]:
sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<PersonKnowsShape> a sh:PropertyShape ;
sh:path :knows ;
sh:targetClass <Person> ;
sh:class <Person> .
""")

dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<Peter>  rdf:type  <Person> ;
        :knows    <Mary> ;
        :knows    [ :name  "Mary" ] ;
        :knows    [ rdf:type  <Person> ] .

<Jane>  :knows  <Jim> .
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/properties/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Peter> ;
            sh:resultMessage "Value does not have class <http://example.org/entities/Person>" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape <http://example.org/entities/PersonKnowsShape> ;
            sh:value <http://example.org/entities/Mary> ],
        [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Peter> ;
            sh:resultMessage "Value does not have class <http://example.org/entities/Person>" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violat

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/Peter>,:knows,<http://example.org/entities/Mary>,sh:ClassConstraintComponent,<http://example.org/entities/PersonKnowsShape>,Value does not have class <http://example.org/...
1,<http://example.org/entities/Peter>,:knows,n4f6c1a5eb5dd4aceb7e56977a80d09bdb1,sh:ClassConstraintComponent,<http://example.org/entities/PersonKnowsShape>,Value does not have class <http://example.org/...


# Task 4

Jede Person hat genau ein Alter (:age). Eine Person ist mindestens 0 und maximal 150 Jahre alt. Erwachsene (Klasse Adult) sind mindestens 19 Jahre alt. Senioren sind mindestens 65 Jahre alt. Kinder sind maximal 12 Jahre alt.
Verwenden Sie sh:minInclusive und sh:maxInclusive (siehe SHACL Recommendation).




In [21]:
sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<PersonAgeShape> a sh:NodeShape ;
sh:targetClass <Person> ;
sh:property [ sh:path :age ; sh:minCount 1 ; sh:maxCount 1 ;  sh:minInclusive "0"^^xsd:integer ; sh:maxInclusive "150"^^xsd:integer ]  .

<AdultAgeShape> a sh:NodeShape ;
sh:targetClass <Adult> ;
sh:class <Adult> ;
sh:property [ sh:path :age ; sh:minInclusive "19"^^xsd:integer ] .

<SeniorAgeShape> a sh:NodeShape ;
sh:targetClass <Senior> ;
sh:property [ sh:path :age ; sh:minInclusive "65"^^xsd:integer ] .

<KidAgeShape> a sh:NodeShape ;
sh:targetClass <Child> ;
sh:property [ sh:path :age ; sh:maxInclusive "12"^^xsd:integer ] .

""")

dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<ABC>   :age    -1 , 27 , 197 .

<John>  rdf:type  <Person> ;
        :age      151 .

<Adult>  rdfs:subClassOf  <Person> .

<Beny>  rdf:type  <Child> ;
        :age      13 .

<Zoe>   rdf:type  <Child> ;
        :age      12 .

<Mary>  rdf:type  <Woman> ;
        :age      17 .

<Bibi>  rdf:type  <Person> ;
        :age      0 .

<Peter>  rdf:type  <Person> .

<Senior>  rdfs:subClassOf  <Adult> .

<Jane>  rdf:type  <Woman> ;
        :age      67 .

<Lili>  rdf:type  <Person> ;
        :age      -1 .

<Senior1>  rdf:type  <Senior> ;
        :age      65 .

<Child>  rdfs:subClassOf  <Person> .

<Bibo>  rdf:type  <Person> ;
        :age      34 , 12 .

<Woman>  rdfs:subClassOf  <Adult> .

<Senior2>  rdf:type  <Senior> ;
        :age      64 .
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/properties/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Mary> ;
            sh:resultMessage "Value is not >= Literal(\"19\", datatype=xsd:integer)" ;
            sh:resultPath :age ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:MinInclusiveConstraintComponent ;
            sh:sourceShape [ sh:minInclusive 19 ;
                    sh:path :age ] ;
            sh:value 17 ],
        [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Senior2> ;
            sh:resultMessage "Value is not >= Literal(\"65\", datatype=xsd:integer)" ;
            sh:resultPath :age ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:MinInclusiveConstraintComponent ;
       

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/Beny>,:age,13,sh:MaxInclusiveConstraintComponent,n3f23791b1fcb400c9a9de9dac1c072edb4,"Value is not <= Literal(""12"", datatype=xsd:int..."
1,<http://example.org/entities/Mary>,:age,17,sh:MinInclusiveConstraintComponent,n3f23791b1fcb400c9a9de9dac1c072edb2,"Value is not >= Literal(""19"", datatype=xsd:int..."
2,<http://example.org/entities/Peter>,:age,None,sh:MinCountConstraintComponent,n3f23791b1fcb400c9a9de9dac1c072edb1,Less than 1 values on <http://example.org/enti...
3,<http://example.org/entities/Lili>,:age,-1,sh:MinInclusiveConstraintComponent,n3f23791b1fcb400c9a9de9dac1c072edb1,"Value is not >= Literal(""0"", datatype=xsd:inte..."
4,<http://example.org/entities/Bibo>,:age,None,sh:MaxCountConstraintComponent,n3f23791b1fcb400c9a9de9dac1c072edb1,More than 1 values on <http://example.org/enti...
5,<http://example.org/entities/John>,:age,151,sh:MaxInclusiveConstraintComponent,n3f23791b1fcb400c9a9de9dac1c072edb1,"Value is not <= Literal(""150"", datatype=xsd:in..."
6,<http://example.org/entities/Senior2>,:age,64,sh:MinInclusiveConstraintComponent,n3f23791b1fcb400c9a9de9dac1c072edb3,"Value is not >= Literal(""65"", datatype=xsd:int..."


# Task 5

Jede Person hat maximal einen Namen. Der Name muss ein Literal sein und vom Datentyp String. Personen dürfen abgesehen von :name und rdf:type keine weiteren Eigenschaften haben.

In [22]:
sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<PersonShape> a sh:NodeShape ;
sh:targetClass <Person> ;
sh:closed true ;
sh:ignoredProperties ( rdf:type ) ;
sh:property [ sh:path :name ; sh:maxCount 1 ; sh:datatype xsd:string ; sh:nodeKind sh:Literal] .
""")

dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<John>  rdf:type  <MalePerson> ;
        :knows    <Mary> .

<FemalePerson>  rdfs:subClassOf  <Person> .

<Mary>  rdf:type  <FemalePerson> ;
        :name     "Mary" .

<Bibi>  rdf:type  <Person> ;
        :name     "Jim" , "Bibi" .

<Jane>  rdf:type  <Person> ;
        :name     2343 .

<Jim>   rdf:type  <Person> ;
        :name     <Jim> .

<MalePerson>  rdfs:subClassOf  <Person> .
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/properties/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Jim> ;
            sh:resultMessage "Value is not Literal with datatype xsd:string" ;
            sh:resultPath :name ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:DatatypeConstraintComponent ;
            sh:sourceShape _:nfa2892a228df442a92741fb9537cf0f6b2 ;
            sh:value <http://example.org/entities/Jim> ],
        [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Jim> ;
            sh:resultMessage "Value is not of Node Kind sh:Literal" ;
            sh:resultPath :name ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NodeKindConstraintComponent ;
            sh:sourceShape _:nfa

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/John>,:knows,<http://example.org/entities/Mary>,sh:ClosedConstraintComponent,<http://example.org/entities/PersonShape>,Node <http://example.org/entities/John> is clo...
1,<http://example.org/entities/Jim>,:name,<http://example.org/entities/Jim>,sh:DatatypeConstraintComponent,nfa2892a228df442a92741fb9537cf0f6b2,Value is not Literal with datatype xsd:string
2,<http://example.org/entities/Jim>,:name,<http://example.org/entities/Jim>,sh:NodeKindConstraintComponent,nfa2892a228df442a92741fb9537cf0f6b2,Value is not of Node Kind sh:Literal
3,<http://example.org/entities/Jane>,:name,2343,sh:DatatypeConstraintComponent,nfa2892a228df442a92741fb9537cf0f6b2,Value is not Literal with datatype xsd:string
4,<http://example.org/entities/Bibi>,:name,None,sh:MaxCountConstraintComponent,nfa2892a228df442a92741fb9537cf0f6b2,More than 1 values on <http://example.org/enti...


# Task 6

Städte sind via Property :inCountry Ländern zugeordnet. Europäische Städte sind europäischen Ländern zugeordnet. Österreichische Städte sind Austria zugeordnet.

In [23]:
sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<CityShape> a sh:NodeShape ;
sh:targetClass <City> ;
sh:property [ sh:path :inCountry ; sh:class <Country> ] .

<EuropeanCity> a sh:NodeShape ;
sh:targetClass <EuropeanCity> ;
sh:property [ sh:path :inCountry ; sh:class <EuropeanCountry> ] .

<AustrianCity> a sh:NodeShape ;
sh:targetClass <AustrianCity> ;
sh:property [ sh:path :inCountry ; sh:in ( <Austria> ) ] .
""")

dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<Paris>  rdf:type   <EuropeanCity> ;
        :inCountry  <France> .

<AustrianCity>  rdfs:subClassOf  <EuropeanCity> .

<Salzburg>  rdf:type  <AustrianCity> ;
        :inCountry  <Germany> .

<EuropeanCountry>  rdfs:subClassOf  <Country> .

<NewYork>  rdf:type  <City> ;
        :inCountry  <USA> .

<Germany>  rdf:type  <EuropeanCountry> .

<EuropeanCity>  rdfs:subClassOf  <City> .

<Vienna>  rdf:type  <AustrianCity> ;
        :inCountry  <Austria> .

<Austria>  rdf:type  <EuropeanCountry> .

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/properties/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/NewYork> ;
            sh:resultMessage "Value does not have class <http://example.org/entities/Country>" ;
            sh:resultPath :inCountry ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape _:nc2fec2a918fa491ca8da763dbb3320f2b1 ;
            sh:value <http://example.org/entities/USA> ],
        [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Paris> ;
            sh:resultMessage "Value does not have class <http://example.org/entities/Country>" ;
            sh:resultPath :inCountry ;
            sh:resultSeverity sh

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/Paris>,:inCountry,<http://example.org/entities/France>,sh:ClassConstraintComponent,nc2fec2a918fa491ca8da763dbb3320f2b2,Value does not have class <http://example.org/...
1,<http://example.org/entities/NewYork>,:inCountry,<http://example.org/entities/USA>,sh:ClassConstraintComponent,nc2fec2a918fa491ca8da763dbb3320f2b1,Value does not have class <http://example.org/...
2,<http://example.org/entities/Paris>,:inCountry,<http://example.org/entities/France>,sh:ClassConstraintComponent,nc2fec2a918fa491ca8da763dbb3320f2b1,Value does not have class <http://example.org/...
3,<http://example.org/entities/Salzburg>,:inCountry,<http://example.org/entities/Germany>,sh:InConstraintComponent,nc2fec2a918fa491ca8da763dbb3320f2b3,Value <http://example.org/entities/Germany> no...


# Task 7
Die Property :worksFor darf nur Personen (als Subjekt) und Organisation (als Objekt) miteinander verbinden.
Hinweis: Lösen Sie die Aufgabe ohne Property Shapes.

In [24]:
sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<WorksForSubjectsShape> a sh:NodeShape;
sh:targetSubjectsOf :worksFor ;
sh:class <Person> .

<WorksForObjectsShape> a sh:NodeShape ;
sh:targetObjectsOf :worksFor ;
sh:class <Organization> .
""")

dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<Mary>  :worksFor  <JKU> .

<Jane>  rdf:type   <Person> ;
        :worksFor  <LMU> .

<Jim>   rdf:type   <Man> ;
        :worksFor  <LMU> .

<University>  rdfs:subClassOf  <Organization> .

<Bob>   rdf:type   <Person> ;
        :worksFor  [ rdf:type  <Organization> ] .

<JKU>   rdf:type  <University> .

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Jim> ;
            sh:resultMessage "Value does not have class <http://example.org/entities/Person>" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape <http://example.org/entities/WorksForSubjectsShape> ;
            sh:value <http://example.org/entities/Jim> ],
        [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/LMU> ;
            sh:resultMessage "Value does not have class <http://example.org/entities/Organization>" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape <http://example.org/entities/WorksForObjectsShape> ;
        

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/Mary>,None,<http://example.org/entities/Mary>,sh:ClassConstraintComponent,<http://example.org/entities/WorksForSubjectsS...,Value does not have class <http://example.org/...
1,<http://example.org/entities/Jim>,None,<http://example.org/entities/Jim>,sh:ClassConstraintComponent,<http://example.org/entities/WorksForSubjectsS...,Value does not have class <http://example.org/...
2,<http://example.org/entities/LMU>,None,<http://example.org/entities/LMU>,sh:ClassConstraintComponent,<http://example.org/entities/WorksForObjectsSh...,Value does not have class <http://example.org/...


# Task 8

Peter kennt ausschließlich Männer, die in Österreich leben.

In [25]:
sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<PeterShape> a sh:NodeShape ;
sh:targetNode <Peter> ;
sh:property [ sh:path :knows ; sh:node <PeterKnowsShape> ] .

<PeterKnowsShape> a sh:NodeShape ;
sh:class <Man> ;
sh:property [ sh:path :livesIn ; sh:minCount 1 ; sh:in ( <Austria> ) ] .
""")

dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<John>  rdf:type  <Man> ;
        :livesIn  <Austria> .

<Mary>  rdf:type  <Woman> ;
        :knows    <Jim> ;
        :livesIn  <Austria> .

<Hans>  rdf:type  <Man> .

<Peter>  :knows  <Josef> , <Hans> , <Franz> , <John> , <Mary> .

<Franz>  rdf:type  <Man> ;
        :livesIn  <Germany> .

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/properties/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Peter> ;
            sh:resultMessage "Value does not conform to Shape <http://example.org/entities/PeterKnowsShape>" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NodeConstraintComponent ;
            sh:sourceShape _:ncf47276708d24bceb7f21489a31e54a7b1 ;
            sh:value <http://example.org/entities/Mary> ],
        [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Peter> ;
            sh:resultMessage "Value does not conform to Shape <http://example.org/entities/PeterKnowsShape>" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstrain

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/Peter>,:knows,<http://example.org/entities/Josef>,sh:NodeConstraintComponent,ncf47276708d24bceb7f21489a31e54a7b1,Value does not conform to Shape <http://exampl...
1,<http://example.org/entities/Peter>,:knows,<http://example.org/entities/Franz>,sh:NodeConstraintComponent,ncf47276708d24bceb7f21489a31e54a7b1,Value does not conform to Shape <http://exampl...
2,<http://example.org/entities/Peter>,:knows,<http://example.org/entities/Hans>,sh:NodeConstraintComponent,ncf47276708d24bceb7f21489a31e54a7b1,Value does not conform to Shape <http://exampl...
3,<http://example.org/entities/Peter>,:knows,<http://example.org/entities/Mary>,sh:NodeConstraintComponent,ncf47276708d24bceb7f21489a31e54a7b1,Value does not conform to Shape <http://exampl...


# Task 9
NOT SOLVED!



In [26]:
sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<NetWeightShape>
	a sh:NodeShape ;
	sh:targetClass <Product> ;
	sh:property [
		sh:path :netWeight ;
		sh:lessThanOrEquals :grossWeight
	].

""")

dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<Cookies>  rdf:type   <Product> ;
        :grossWeight  0.2 ;
        :netWeight    0.12 .

<Milk>  rdf:type      <Product> ;
        :grossWeight  1.1 ;
        :netWeight    1 .

<Peter>  rdf:type     <Person> ;
        :grossWeight  74 ;
        :netWeight    72 .

<Bread>  rdf:type     <Product> ;
        :grossWeight  1.1 ;
        :netWeight    1.2 .

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/properties/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Bread> ;
            sh:resultMessage "Value of <http://example.org/entities/Bread>->:grossWeight < Literal(\"1.2\", datatype=xsd:decimal)" ;
            sh:resultPath :netWeight ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:LessThanOrEqualsConstraintComponent ;
            sh:sourceShape [ sh:lessThanOrEquals :grossWeight ;
                    sh:path :netWeight ] ;
            sh:value 1.2 ] .




,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/Bread>,:netWeight,1.2,sh:LessThanOrEqualsConstraintComponent,ne72e0d475d5e4b36ae5045754fd7fcccb1,Value of <http://example.org/entities/Bread>->...


# Task 10
Die Objekte von knows-Statements haben einen Namen (Property :name) oder haben einen Vornamen (:givenName) und einen Nachnamen (:familyName). eine

In [27]:
sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<ObjectOfKnowsShape> a sh:NodeShape ;
sh:targetObjectsOf :knows ;
sh:or (
        [
            sh:path :name ;
            sh:minCount 1 ;
            sh:maxCount 1 ;
        ]
        [
            sh:and (
                [
                    sh:path :givenName ;
                    sh:minCount 1 ;
                    sh:maxCount 1 ;
                ]
                [
                    sh:path :familyName ;
                    sh:minCount 1 ;
                    sh:maxCount 1 ;
                ]
            )
         ]
      ).
""")

dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<Sara>  :givenName  "Sarah" .

<John>  :familyName  "Black" .

<Mary>  :knows  <Pete> .

<Bob>   :familyName  "Builder" ;
        :givenName   "Bob" .

<Pete>  :knows  <Bob> , <Sara> , <John> ;
        :name   "Peter Parker" .

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Sara> ;
            sh:resultMessage "Node <http://example.org/entities/Sara> does not conform to one or more shapes in [ sh:maxCount Literal(\"1\", datatype=xsd:integer) ; sh:minCount Literal(\"1\", datatype=xsd:integer) ; sh:path :name ] , [ sh:and ( [ sh:maxCount Literal(\"1\", datatype=xsd:integer) ; sh:minCount Literal(\"1\", datatype=xsd:integer) ; sh:path :givenName ] [ sh:maxCount Literal(\"1\", datatype=xsd:integer) ; sh:minCount Literal(\"1\", datatype=xsd:integer) ; sh:path :familyName ] ) ]" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:OrConstraintComponent ;
            sh:sourceShape <http://example.org/entities/ObjectOfKnowsShape> ;
            sh:value <http://example.org/entities/Sara> ]

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/Sara>,None,<http://example.org/entities/Sara>,sh:OrConstraintComponent,<http://example.org/entities/ObjectOfKnowsShape>,Node <http://example.org/entities/Sara> does n...
1,<http://example.org/entities/John>,None,<http://example.org/entities/John>,sh:OrConstraintComponent,<http://example.org/entities/ObjectOfKnowsShape>,Node <http://example.org/entities/John> does n...
